# Import Library

In [1]:
import pandas as pd

## Import files

In [2]:
#Dimension Table
questionSets = pd.read_excel(r"C:\Users\kanya\Downloads\Dataset Survey Dictionary.xlsx", sheet_name="Question Sets")
questions = pd.read_excel(r"C:\Users\kanya\Downloads\Dataset Survey Dictionary.xlsx", sheet_name="Questions")
question_mapping = pd.read_excel(r"C:\Users\kanya\Downloads\Dataset Survey Dictionary.xlsx", sheet_name="QuestionMapping")
column_mapping = pd.read_excel(r"D:\ThaiPBS\Dataset\Column_Mapping.xlsx", sheet_name="ColumnMapping")
choices = pd.read_excel(r"C:\Users\kanya\Downloads\Dataset Survey Dictionary.xlsx", sheet_name="Choices")

In [3]:
#Raw Data
df = pd.read_excel(r"C:\Users\kanya\Downloads\raw3dataset\Targeted_Documentary2025_Raw data.xlsx", sheet_name="Raw_coded_24")  #change path here

# Clean Data

In [4]:
# Set QuestionSetID for filter the column_mapping
current_qs = "QS8"

# Filter ColumnMap for current QuestionSet ===
map_filtered = column_mapping[column_mapping['QuestionSetID'] == current_qs]
# clean blank
#map_filtered = map_filtered.dropna(subset=['OriginalName', 'NewName'])

rename_dict = dict(zip(map_filtered['OriginalName'], map_filtered['NewName'])) # Create rename dictionary eg. [('q1': 'Age'), ('q2': 'Gender')]

# Delete unuse column (original name)
#global_issue_data = global_issue_data.drop('q24', axis=1)
#global_issue_data = global_issue_data.drop(['q3','q4'], axis=1)

# Rename columns dynamically ===
data_renamed = df.rename(columns=rename_dict)

# Drop NaN columns
data_renamed = data_renamed.loc[:, data_renamed.columns.notna()]

In [5]:
data_renamed.head()

,Q165,Q166,Q167,Q169,Q162,Q184,Q175,Q179,Q249,Q174,...,Q598A10,Q598A11,Q14text8,Q15text7,Q228text5,Q229text5,Q230text9,Q232text7,Q232text7,Q232text7
0,2,40,3,2,1,5,1,1.0,5,4,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,54,4,3,1,5,1,19.0,6,5,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,22,1,1,1,8,8,NaN,3,4,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,33,2,2,1,8,1,26.0,10,4,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,51,4,3,3,8,11,NaN,1,7,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Unpivot data

In [6]:
# Unpivot data
id_col = 'Audience ID'
data_renamed[id_col] = [f"AU_QS8_{i}" for i in range(1, len(data_renamed) + 1)] # Modify prefix to QuestionSet
unpivoted = data_renamed.melt(id_vars=[id_col], var_name="Attribute", value_name="Value")
unpivoted = unpivoted.reset_index(drop=True)
unpivoted['Index'] = unpivoted.index + 1
unpivoted['RID'] = "RID" + unpivoted['Index'].astype(str)


In [7]:
unpivoted.head(10)

,Audience ID,Attribute,Value,Index,RID
0,AU_QS8_1,Q165,2,1,RID1
1,AU_QS8_2,Q165,2,2,RID2
2,AU_QS8_3,Q165,2,3,RID3
3,AU_QS8_4,Q165,2,4,RID4
4,AU_QS8_5,Q165,2,5,RID5
5,AU_QS8_6,Q165,2,6,RID6
6,AU_QS8_7,Q165,2,7,RID7
7,AU_QS8_8,Q165,1,8,RID8
8,AU_QS8_9,Q165,2,9,RID9
9,AU_QS8_10,Q165,2,10,RID10


In [8]:
unpivoted[unpivoted['Attribute'] == 'Q238text8']

,Audience ID,Attribute,Value,Index,RID


# Tranform Data

In [9]:
# create QuestionID
def extract_qid(attr):
    #สร้าง QID ให้ A คือ ชื่อChoiceID ให้รู้ว่าเป็น multiple choice / code คือ opentext-Code / text คือ opentext-Text / rank คือ คำตอบของอันดับนั้นๆ
    for key in ["A", "code", "text","rank"]: 
        if key in attr:
            return attr.split(key)[0]
    return attr

unpivoted['QuestionID'] = unpivoted['Attribute'].apply(extract_qid)

In [10]:
unpivoted[unpivoted['QuestionID'] == 'Q529']

,Audience ID,Attribute,Value,Index,RID,QuestionID


## Merge Data

In [11]:
# Merge with Questions เพื่อใช้ดู AnswerType
data = unpivoted.merge(questions, how="left", on="QuestionID")

In [12]:
IsOpenText_QID = choices.loc[choices['IsOpenText'] == 'Yes']['QuestionID'].unique().tolist()
#print(IsOpenText_QID)
#['Q175', 'Q14', 'Q67', 'Q69', 'Q158', 'Q238', 'Q247', 'Q250', 'Q253', 'Q279', 'Q281', 'Q301']

In [13]:
data.loc[data['QuestionID'] == 'Q238',:]

,Audience ID,Attribute,Value,Index,RID,QuestionID,QuestionText,AnswerType,MaxRank,Category,Sub_Category1,Sub_Category2,Category JSON,Condition


In [14]:
# create ChoiceID
def derive_choice(row):
    attr = row['Attribute']
    val = row['Value']
    qid = row['QuestionID']
    if "A" in attr: # เป็น multiple choice อยู่แล้ว
        return attr
    elif qid != "Q166" and pd.notna(val) and "code" not in attr and "text" not in attr: # เลือกคำตอบเดียว ไม่ให้ติดอายุหรือopentextมาด้วย
        try:
            return f"{qid}A{int(float(val))}"  # Convert strings like "3" or 3.0 to 3
        except:
            return None
    elif "text" in attr and qid in IsOpenText_QID : # กรณีเป็น opentext ที่มาจาก choice อื่น ๆ
        try:
            return f"{qid}A{attr.split("text")[1]}"  # ให้ตรงกับ choice ที่เป็นอื่น ๆ
        except:
            return None
    else:
        return f"{qid}A1" # Q166 / opentext

data['ChoiceID'] = data.apply(derive_choice, axis=1)

In [15]:
data[data['QuestionID'] == 'Q528']

,Audience ID,Attribute,Value,Index,RID,QuestionID,QuestionText,AnswerType,MaxRank,Category,Sub_Category1,Sub_Category2,Category JSON,Condition,ChoiceID


In [16]:
#data[(data['Audience ID'] == 'AU_QS8_5') & (data['QuestionID'] == 'Q232')]

In [17]:
# Delete blank value
# Only drop rows with nulls in 'Value' where AnswerType does NOT contain 'opentext' (case-insensitive)
#data = data.loc[~(data['Value'].isnull() & ~data['AnswerType'].astype(str).str.contains('opentext', case=False, na=False))]
data = data.loc[~(data['Value'].isnull())]


In [18]:
# Add NumericValue for Rating
    
data['NumericValue'] = data.apply(lambda r: r['Value'] if pd.notnull(r['AnswerType']) and 'ให้คะแนน' in r['AnswerType']
                                  #(r['AnswerType'] == 'ตัวเลข' or 'ให้คะแนน' in r['AnswerType']) 
                                  else None, axis=1)

In [19]:
#data[data['QuestionID'] == 'Q305']

In [20]:
# Add OpentextValue, TextType (*ทุก int ใน opentext จะกลายเป็น code ต้องระวัง)
data['OpentextValue'] = data.apply(lambda r: r['Value'] if isinstance(r['Value'], str) or 'code' in r['Attribute'] or 'text' in r['Attribute'] else None, axis=1)
#data.apply(lambda r: r['Value'] if isinstance(r['Value'], str))
def determine_opentext_type(x):
    if pd.notnull(x['OpentextValue']):
        if 'text' in x['Attribute']:
            return 'Text'
        elif 'code' in x['Attribute']:
            return 'Code'
    return None

data['OpentextType'] = data.apply(determine_opentext_type, axis=1)

# Drop value Null (in case code is null)
data = data[~((data['AnswerType'].astype(str).str.contains('opentext', case=False, na=False)) & 
              (data['OpentextValue'].isnull()) & data['ChoiceID'].isnull())] # In case it's both choice and opentext

In [21]:
# Add Selected for MultipleChoice
data['Selected'] = data.apply(lambda r: r['Value'] if 'เลือกหลายคำตอบ' in str(r['AnswerType']) and not "text" in r['Attribute'] else None, axis=1)


In [22]:
#data[data['QuestionID'] == 'Q602']

In [23]:
# Add RankValue using MaxRank ===
def assign_rank(r):
    if isinstance(r['AnswerType'], str) and 'จัดอันดับ' in r['AnswerType'] \
    and pd.notnull(r['MaxRank']) and pd.notnull(r['Value']) and not "text" in r['Attribute']:
        return (r['MaxRank'] - r['Value']) + 1 if r['Value'] <= r['MaxRank'] else None
    elif 'rank' in r['Attribute']:
        return r['Attribute'].split('rank')[1]
    return None

data['RankValue'] = data.apply(assign_rank, axis=1)

In [24]:
# Add BirthdayValue and Age
from datetime import datetime

# Check if any row has QuestionID == "Q166"
has_q166 = data['QuestionID'].eq("Q166").any()

def assign_age(r):
    if has_q166 and pd.notnull(r['Value']) and r['QuestionID'] == 'Q166':
        if isinstance(r['Value'],int) or r['Value'].isdigit():
            return int(r['Value'])
        else:
            #current_year = datetime.now().year
            year = datetime.strptime(r['Value'], "%d/%m/%Y").year
            return 2024 - year                                   #ปีที่เก็บแบบสอบถาม
        
def assign_birthday(r):
    if has_q166 and pd.notnull(r['Value']) and r['QuestionID'] == 'Q166':
        if isinstance(r['Value'],int) or r['Value'].isdigit():
            year = 2024 - int(r['Value'])                        #ปีที่เก็บแบบสอบถาม
            return pd.Timestamp(year=year, month=1, day=1)
        else:
            return r['Value']

'''
# Create Age column
data['Age'] = data.apply(
    lambda r: r['Value'] if r['QuestionID'] == 'Q166' and pd.notnull(r['Value']) else None,
    axis=1
)

# p53 lookup dictionary: get p53 value(birthday) into Q166
p53_lookup = {}
if has_q166:
    p53_lookup = data.loc[data['QuestionID'] == 'P53'].set_index('Audience ID')['Value'].to_dict()

def assign_birthday(r):
    if has_q166 and pd.notnull(r['Value']) and r['QuestionID'] == 'Q166':
        p53_value = p53_lookup.get(r['Audience ID'])
        if pd.notnull(p53_value):
            return p53_value
    elif not has_q166 and pd.notnull(r['Age']):
        #current_year = datetime.now().year
        birth_year = 2024 - int(r['Age'])
        return pd.Timestamp(year=birth_year, month=1, day=1)
    else:
        return None
'''

data['Age'] = data.apply(assign_age, axis=1)
data['Birthday'] = data.apply(assign_birthday, axis=1)
data['Age'] = data['Age'].fillna(0).astype(int)


In [25]:
#data[data['QuestionID'] == 'Q166']

## Re-order Columns

In [26]:
#drop_columns = ['Attribute', 'Value', 'AnswerType', 'MaxRank', 'Index']
columns = ['RID', 'Audience ID', 'QuestionID', 'ChoiceID','Age','Birthday',"NumericValue",
           'OpentextValue','OpentextType', 'Selected', 'RankValue']
#data = data.drop(columns=drop_columns)
final = data[columns]

In [27]:
final[final['QuestionID'] == 'Q166']

,RID,Audience ID,QuestionID,ChoiceID,Age,Birthday,NumericValue,OpentextValue,OpentextType,Selected,RankValue
2005,RID2006,AU_QS8_1,Q166,Q166A1,40,1984-01-01,None,None,None,NaN,None
2006,RID2007,AU_QS8_2,Q166,Q166A1,54,1970-01-01,None,None,None,NaN,None
2007,RID2008,AU_QS8_3,Q166,Q166A1,22,2002-01-01,None,None,None,NaN,None
2008,RID2009,AU_QS8_4,Q166,Q166A1,33,1991-01-01,None,None,None,NaN,None
2009,RID2010,AU_QS8_5,Q166,Q166A1,51,1973-01-01,None,None,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...
4005,RID4006,AU_QS8_2001,Q166,Q166A1,21,2003-01-01,None,None,None,NaN,None
4006,RID4007,AU_QS8_2002,Q166,Q166A1,23,2001-01-01,None,None,None,NaN,None
4007,RID4008,AU_QS8_2003,Q166,Q166A1,50,1974-01-01,None,None,None,NaN,None
4008,RID4009,AU_QS8_2004,Q166,Q166A1,21,2003-01-01,None,None,None,NaN,None


In [28]:
final.to_excel(r"D:\ThaiPBS\Dataset\Clean Data 2\Documentary2024.xlsx", index=False)

# Cross-tab

### Single-selected question with Single-selected question example

In [ ]:

# loaded response and choices tables
response = pd.read_excel(r"D:\ThaiPBS\Dataset\Clean Data 2\พฤติกรรมการใช้สื่อของคนแต่ละช่วงวัย2.xlsx", sheet_name="Sheet1")       # cleaned response table
choices = pd.read_excel(r"D:\ThaiPBS\รวมโค้ด\Dataset Survey Dictionary.xlsx", sheet_name="Choices")         # choice lookup table

# Step 1: Filter responses to just Q1 and Q2 หรือ คำถามที่ต้องการวิเคราะห์
q1 = response[response["QuestionID"] == "315"][["Audience ID", "ChoiceID"]].rename(columns={"ChoiceID": "Q315_ChoiceID"})
q2 = response[response["QuestionID"] == "290"][["Audience ID", "ChoiceID"]].rename(columns={"ChoiceID": "Q290_ChoiceID"})

# Step 2: Merge Questions responses by Audience ID
merged = pd.merge(q1, q2, on="Audience ID", how="inner")

# Step 3: Map ChoiceText from Choices table
choices_q1 = choices[choices["QuestionID"] == "Q315"][["ChoiceID", "ChoiceText"]].rename(columns={
    "ChoiceID": "Q1_ChoiceID", "ChoiceText": "Q1_ChoiceText"
})
choices_q2 = choices[choices["QuestionID"] == "Q290"][["ChoiceID", "ChoiceText"]].rename(columns={
    "ChoiceID": "Q2_ChoiceID", "ChoiceText": "Q2_ChoiceText"
})

merged = pd.merge(merged, choices_q1, on="Q1_ChoiceID", how="left")
merged = pd.merge(merged, choices_q2, on="Q2_ChoiceID", how="left")
"""
# Step 4: Create pivot table (cross-tab)
pivot = pd.pivot_table(
    merged,
    values="Audience ID",
    index="Q1_ChoiceText",
    columns="Q2_ChoiceText",
    aggfunc="nunique",
    fill_value=0
)

# Show the result
print(pivot)
"""
# aggfunc="nunique", "count", "sum", "mean", etc.


'\ndef extract_choice_suffix(choice_id):\n    if isinstance(choice_id, str) and "A" in choice_id:\n        return choice_id.split("A")[-1]\n    return None\n\nq1["ChoiceSuffix"] = q1["ChoiceID"].apply(extract_choice_suffix)\nq2["ChoiceSuffix"] = q2["ChoiceID"].apply(extract_choice_suffix)\n\n# Now merge on Audience ID + ChoiceSuffix\nmerged = pd.merge(q1, q2, on=["Audience ID", "ChoiceSuffix"])\n'

### Multiple choice question with rank or rating example

In [ ]:
"""
def extract_choice_suffix(choice_id):
    if isinstance(choice_id, str) and "A" in choice_id:
        return choice_id.split("A")[-1]
    return None

q1["ChoiceSuffix"] = q1["ChoiceID"].apply(extract_choice_suffix)
q2["ChoiceSuffix"] = q2["ChoiceID"].apply(extract_choice_suffix)

# Now merge on Audience ID + ChoiceSuffix
merged = pd.merge(q1, q2, on=["Audience ID", "ChoiceSuffix"])
"""

In [ ]:
"""
# === Step 6: Derive QuestionID ===
def extract_qid(attr):
    for key in ["A", "code", "text"]:
        if key in attr:
            return attr.split(key)[0]
    return attr

unpivoted['QuestionID'] = unpivoted['Attribute'].apply(extract_qid)

# === Step 7: Derive ChoiceID ===
def derive_choice(row):
    attr = row['Attribute']
    val = row['Value']
    qid = row['QuestionID']
    if "A" in attr:
        return attr
    elif qid != "P7" and pd.api.types.is_number(val) and "code" not in attr:
        return f"{qid}A{int(val)}"
    else:
        return None

unpivoted['ChoiceID'] = unpivoted.apply(derive_choice, axis=1)

# === Step 8: Merge with Questions ===
data = unpivoted.merge(questions, how="left", on="QuestionID")

# === Step 9: Add OpentextValue, TextType ===
data['OpentextValue'] = data.apply(lambda r: r['Value'] if isinstance(r['Value'], str) or 'code' in r['Attribute'] else None, axis=1)
data['OpentextType'] = data['OpentextValue'].apply(lambda x: "Text" if isinstance(x, str) else ("Code" if pd.notnull(x) else None))

# === Step 10: Add Selected for MultipleChoice ===
data['Selected'] = data.apply(lambda r: r['Value'] if r['AnswerType'] == 'MultipleChoice' else None, axis=1)

# === Step 11: Add RankValue using MaxRank ===
def assign_rank(r):
    if r['AnswerType'] == 'Ranking' and pd.notnull(r['MaxRank']) and pd.notnull(r['Value']):
        return r['Value'] if r['Value'] <= r['MaxRank'] else None
    return None

data['RankValue'] = data.apply(assign_rank, axis=1)

# === Final Columns ===
columns = ['Audience ID', 'RID', 'Index', 'QuestionID', 'ChoiceID', 'Attribute', 'Value',
           'OpentextValue', 'OpentextType', 'Selected', 'RankValue', 'AnswerType', 'MaxRank']

final = data[columns]

# === Export result ===
final.to_excel("cleaned_survey.xlsx", index=False)
print("✅ Survey data transformed and saved to cleaned_survey.xlsx")
"""

'\n# === Step 6: Derive QuestionID ===\ndef extract_qid(attr):\n    for key in ["A", "code", "text"]:\n        if key in attr:\n            return attr.split(key)[0]\n    return attr\n\nunpivoted[\'QuestionID\'] = unpivoted[\'Attribute\'].apply(extract_qid)\n\n# === Step 7: Derive ChoiceID ===\ndef derive_choice(row):\n    attr = row[\'Attribute\']\n    val = row[\'Value\']\n    qid = row[\'QuestionID\']\n    if "A" in attr:\n        return attr\n    elif qid != "P7" and pd.api.types.is_number(val) and "code" not in attr:\n        return f"{qid}A{int(val)}"\n    else:\n        return None\n\nunpivoted[\'ChoiceID\'] = unpivoted.apply(derive_choice, axis=1)\n\n# === Step 8: Merge with Questions ===\ndata = unpivoted.merge(questions, how="left", on="QuestionID")\n\n# === Step 9: Add OpentextValue, TextType ===\ndata[\'OpentextValue\'] = data.apply(lambda r: r[\'Value\'] if isinstance(r[\'Value\'], str) or \'code\' in r[\'Attribute\'] else None, axis=1)\ndata[\'OpentextType\'] = data[\'Ope